In [1]:
#- main machine learning libraries used

import tensorflow
import tensorflow.keras as keras  
import sklearn

#- other libraries used
import os
import pandas as pd
import matplotlib.pyplot as plt 

In [2]:
from datetime import datetime
today = datetime.today()

print('latest run on:', today)

latest run on: 2022-06-05 13:52:56.954083


## check tensorflow version

In [6]:
print (tensorflow.__version__)

2.9.1


## my functions(.) here

## input directory

In [7]:
dataset_dir = "/Users/ehsanmos/MLP_dataset"

In [8]:
if os.path.isdir(dataset_dir) == False:
    print("dataset directory NOT found!")
else:
    print("dataset directory FOUND!")

dataset directory FOUND!


## Load input/ training dataset

before doing this section, process filter final input dataset with "check_n_filter_final_dataset"

In [9]:
in_ds = "atmmodel_april_2016_k_zero_9cams4bands_preprocessed.csv"

## check if input dataset file exists

In [10]:
in_ds_fullpath = os.path.join(dataset_dir, in_ds)
print(in_ds_fullpath)

if (not os.path.isfile(os.path.join(in_ds_fullpath))):
    raise SystemExit()
else:
    print("input dataset found!")

/Users/ehsanmos/MLP_dataset/atmmodel_april_2016_k_zero_9cams4bands_preprocessed.csv
input dataset found!


## Read in dataset and look at dataset columns

In [11]:
df_orig = pd.read_csv(in_ds_fullpath, engine='python')

In [12]:
df_orig.shape

(40775, 15)

In [13]:
df_orig.columns

Index(['firstLat', 'firstLon', 'anr', 'ang', 'anb', 'annir', 'aa', 'af', 'ba',
       'bf', 'ca', 'cf', 'da', 'df', 'rms'],
      dtype='object')

Note: we will build an input dataset with 9 cameras to train the mlp model

In [15]:
#- build dataset with 9 cameras
excluce_columns = ['firstLat', 'firstLon', 'ang', 'anb', 'annir']
input_ds_for_training = df_orig.drop(excluce_columns, axis=1)

print('we are using %s columns in our training dataset:' %len(input_ds_for_training.columns))
print(input_ds_for_training.columns)  # columns should be only 9 cameras + rms 


we are using 10 columns in our training dataset:
Index(['anr', 'aa', 'af', 'ba', 'bf', 'ca', 'cf', 'da', 'df', 'rms'], dtype='object')


## shuffle rows of input dataset

In [16]:
from sklearn.utils import shuffle

input_ds_for_training = shuffle(input_ds_for_training)

## Split dataset to train-test parts for training algorithms
- we devided to plit our dataset to 2 parts (2-part split)
- Here we use the ‘train_test_split’ to split the data in 80:20 ratio i.e. 80% of the data will be used for training the model while 20% will be used for testing the model that is built out of it.
- note: last column should be label == rms

In [17]:
#- split data set to X and Y

X = input_ds_for_training.iloc[:, :-1] # to select up to last column of dataset OR [:, 0:3]
Y = input_ds_for_training.iloc[:, -1:] # to select last column of DF

print(X.shape)
print(Y.shape)

(40775, 9)
(40775, 1)


In [18]:
#- now split dataset to train-test

from sklearn.model_selection import train_test_split

#- we use this function to split data-- from here because we are usiong SKlearn library, we change all data structures from Pandas DF to numpy
# X_train, X_test, y_train, y_test = train_test_split(X.to_numpy(), Y.to_numpy(), test_size=0.2, random_state=123) # Q- input is DF or numpy array?

test_data_size = 0.3
print("test size= %d percent" %(test_data_size*100))
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=test_data_size, random_state=123) # Q- input is DF or numpy array?


print("train:")
print(x_train.shape)
print(y_train.shape)

print("test:")
print(x_test.shape)
print(y_test.shape)


test size= 30 percent
train:
(28542, 9)
(28542, 1)
test:
(12233, 9)
(12233, 1)


Qn- how about train-val-test (3 sections)? is this for DL?

## Feature scalling

change the scale/range of input features from their original range to a new range. So changed features will have mean=0 and std=1.

source: https://www.enjoyalgorithms.com/blog/need-of-feature-scaling-in-machine-learning

- We rescale data after we split data to train-test
- all features have the same range to reduce bias in data 
- perform this step before splitting data into train-test split
- We normalize data using the training data

Qn- why FS is important? why we do FS?

Qn- which method? 

1) standardization/ Z-score/ StandardScaler() == mean=0 & std=1; Standardize features by removing the mean and scaling to unit variance; good for datasets w/ outliers;

2) MinMaxScalar() == Transform features by scaling each feature to a given range (usually [0,1])

3) normalize() == Scale input vectors individually to unit norm (vector length).

source: https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing



### 1) Using MinMaxScaler() method to rescale input features

In [17]:
# #- import necessary libraries for Neural Nets
# from sklearn.preprocessing import MinMaxScaler

# #- fit data

# scaler_x = MinMaxScaler()
# scaler_y = MinMaxScaler()

# scaler_x.fit(X_train) # returns Fitted scaler
# X_train_scaled = scaler_x.transform(X_train)  # transforms data

# scaler_x.fit(X_test)
# X_test_scaled = scaler_x.transform(X_test)

# scaler_y.fit(y_train)
# y_train_scaled = scaler_y.transform(y_train)

# scaler_y.fit(y_test)
# y_test_scaled = scaler_y.transform(y_test)

### 2) Using StandardScaler() method 

to rescale input features to mean of 0 and std of 1

In [19]:
from sklearn.preprocessing import StandardScaler

scaler_x = StandardScaler()
scaler_y = StandardScaler()

x_train_scaled = scaler_x.fit(x_train).transform(x_train) # returns daata w/ mean 0 & std 1
y_train_scaled = scaler_y.fit(y_train).transform(y_train)
x_test_scaled = scaler_x.fit(x_test).transform(x_test)
y_test_scaled = scaler_y.fit(y_test).transform(y_test)

Check types of input dataset data structure; should be 2D arrays, or Pandas DataFrame

In [20]:
# print(type(x_train_scaled))
# print(type(y_train_scaled))
# print(type(x_test_scaled))
# print(type(y_test_scaled))

print(x_train_scaled.mean())
print(x_train_scaled.std())
print(y_train_scaled.mean())
print(y_train_scaled.std())
print(x_test_scaled.mean())
print(x_test_scaled.std())
print(y_test_scaled.mean())
print(y_test_scaled.std())

7.081141255949737e-17
0.881917103688197
1.960452681700928e-16
1.0
1.4666234021043513e-17
0.8819171036881968
4.733853753326916e-17
1.0


In [21]:
x_train_scaled.shape

(28542, 9)

## Neural Network (Regression)

### Building the NN model
Q- how find the best architecture? for mlp?

In [22]:
#- implementation-2
import tensorflow as tf
from tensorflow.keras import layers


# Define Sequential model with 3 layers
mlp_model = tf.keras.Sequential([
    # hidden layers
    layers.Dense(9, input_dim=9, activation="relu", kernel_initializer='normal', name="hidden-layer-1"), # input_dim=3 == input shape will build our model automatically
    layers.Dense(9, activation="relu", kernel_initializer='normal', name="hidden-layer-2"),  # name should be attached; one single word!
    layers.Dense(9, activation="relu", kernel_initializer='normal', name="hidden-layer-3"),  # name should be attached; one single word!

    # output layer
    layers.Dense(1, activation='linear', kernel_initializer='normal', name="output-layer-SIR"),  # linear activation==no activation
    ])

mlp_model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden-layer-1 (Dense)      (None, 9)                 90        
                                                                 
 hidden-layer-2 (Dense)      (None, 9)                 90        
                                                                 
 hidden-layer-3 (Dense)      (None, 9)                 90        
                                                                 
 output-layer-SIR (Dense)    (None, 1)                 10        
                                                                 
Total params: 280
Trainable params: 280
Non-trainable params: 0
_________________________________________________________________


2022-06-05 19:28:35.413311: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### plot the model

In [38]:
from tensorflow.keras.utils import plot_model

plot_model(mlp_model, to_file='model.png', show_shapes=True, show_layer_names=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


### compile and train the model

In [ ]:
#- compile model

mlp_model.compile(loss='mse', optimizer='adam', metrics=['mse','mae'])

In [ ]:
#- train the network

train_history = mlp_model.fit(X_train_scaled, 
                              y_train_scaled, 
                              epochs=50,
                              batch_size=100, 
                              verbose=1, 
                              validation_split=0.2)

### Training error

In [ ]:
print(train_history.history.keys())
print('\n')

# print("-> model: %s" % train_model)
print("Training Loss: %.2f (units?)" % train_history.history['loss'][-1])  # will return the loss in the last training epoch
print("Training MSE: %.2f (cm^2 roughness)" % (train_history.history['mse'][-1]))
print("Training MAE: %.2f (cm roughness)" % (train_history.history['mae'][-1]))


## Plot loss of training

In [ ]:
print(train_history.history.keys())
# "Loss" val==validation==test
plt.plot(train_history.history['loss'])
plt.plot(train_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

### Testing the NN model and report error 

In [ ]:
'''Returns the loss value & metrics values for the model in test mode.
 similar to test'''

test_metrics = mlp_model.evaluate(X_test_scaled, y_test_scaled, batch_size=20, verbose=1) # Returns the loss value & metrics values for the model in test mode


In [ ]:
# print(test_metrics)
import math

print(test_metrics)

mse_test = test_metrics[1]
print("Test RMSE: %.2f (cm roughness)" %math.sqrt(mse_test))
print("Test MAE: %.2f (cm roughness)" %(test_metrics[2]))

### predict on test and scale back resutls and report them

when to use this?

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

y_pred_mlp = mlp_model.predict(X_test)  # Generates output predictions for the input samples


##- scale back the predictions
    
# y_pred_ann_original = scaler_y.inverse_transform(y_pred_ann_scaled)

# print('Test RMSE: %.2f' %math.sqrt(mean_squared_error(y_test, y_pred_ann_original))) # square root of MSE.


### save the trained model

In [ ]:
# save model and architecture to single file
model_name = "trained_model_3L9N_50epoch.h5"  # find a way to rename this online

model_fp = os.path.join(dataset_dir, model_name) 
mlp_model.save(model_fp)